## Tool definition

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [1]:
import os
from dotenv import load_dotenv
from langchain_ibm import ChatWatsonx
load_dotenv()

parameters = {
    "temperature": 0.5,
    "max_tokens": 100,
}

model = ChatWatsonx(
    model_id="meta-llama/llama-4-maverick-17b-128e-instruct-fp8",
    url=os.environ["WATSONX_URL"],
    project_id=os.environ["WATSONX_PROJECT_ID"],
    params=parameters,
)


In [ ]:
from langchain.tools import tool
# provide detailed description 
# for the agent to understand what the function does.
# by default tools name come from the function name
@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

In [3]:
# we can override the default name 
# by a custom function decorator name
@tool("square_root")
def tool1(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

In [4]:
# the doc-string can also be in teh function decorator itself as description
@tool("square_root", description="Calculate the square root of a number")
def tool1(x: float) -> float:
    return x ** 0.5

In [ ]:
# this invocation is exactly what the agent would do
# when calling the tool 
tool1.invoke({"x": 467})

21.61018278497431

## Adding to agents

In [7]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[tool1],
    system_prompt="You are an arithmetic wizard. Use your tools to calculate the square root and square of any number."
)

In [ ]:
agent = create_agent(
    model=model,
    tools=["square_root"],
    system_prompt="You are an arithmetic wizard. Use your tools to calculate the square root and square of any number."
)

In [8]:
from langchain.messages import HumanMessage


question = HumanMessage(content="What is the square root of 467?")

response = agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

√467 ≈ 21.61018278497431


In [ ]:
from pprint import pprint

pprint(response['messages'])

# NOTE: AIMessage(content='" is empty. however, the tool call can be traced"
# tool_calls=[{'name': 'square_root', 'args': {'x': 467}
# the tool output can be seen in tool message 
# ToolMessage(content='21.61018278497431', name='square_root'
# and finally the AI message
#  AIMessage(content='√467 ≈ 21.61018278497431'

[HumanMessage(content='What is the square root of 467?', additional_kwargs={}, response_metadata={}, id='a89d9f2d-ee01-4a71-acf3-9cc2d1c20e11'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1176, 'prompt_tokens': 158, 'total_tokens': 1334, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1152, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpKg285WqZF6D9klnFS5ktxlpl2z0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b42ad-d9ef-7d02-bfd8-33cf09902582-0', tool_calls=[{'name': 'square_root', 'args': {'x': 467}, 'id': 'call_zOA9XQiILkEYthARk9h8H0Zz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 158, 'output_tokens': 1176, 'total_tokens': 1334, 'input_

In [10]:
print(response["messages"][1].tool_calls)

[{'name': 'square_root', 'args': {'x': 467}, 'id': 'call_zOA9XQiILkEYthARk9h8H0Zz', 'type': 'tool_call'}]
